# Compound Bow


## Imports

In [1]:
from sympy import *
from sympy.physics.mechanics import *

## Symbols

The retract distance for the bow is given by $x$, and the forward speed by $v$. Thus, the single generalized coordinate is $x$, and the corresponding generalized speed is $v$. Because the positive direction of the two is opposite, we define the kinematic equation $\dot{x} = -v$, or by the convention used in `sympy`, $\dot{x}+v=0$.

In [2]:
x, v = dynamicsymbols('x, v')
kd = [diff(x) + v]

We will define the origin of the model as a point at the grip of the bow. The line is initially set back a constant distance $x_0$ from this point, at which point forces are in equilibrium.

In [3]:
x0 = symbols('x_0')

The limbs of the bow will be represented by rigid bodies with a rotational spring attached at fixed positions $(x_l, y_l)$ and $(x_l, -y_l)$ relative to the origin. The length of each limb is $l$, and the undeflected angle relative to the horizontal is $\theta_0$. The effective angular stiffness of each limb is $k$. The mass and moment of inertia for each limb are $m_l$ and $I_l$, rsspectively.

In [4]:
xl, yl, l, theta0, ml, Il = symbols('x_l, y_l, l, \\theta_0, m_l, I_l')

The extent of the limb's angular displacement relative to $\theta_0$ is $\theta$. 

In [5]:
theta = dynamicsymbols('\\theta')

The inner and outer radii at the cam are defined as $r$ and $d$, respectively. These are initially defined as constants, though they may alternately be defined as functions of angular displacement for non-circular geometries.

In [6]:
r, d = symbols('r, d')

The angular displacement for the cams relative to their initial angle is $\phi$.

In [7]:
phi = dynamicsymbols('\phi')